<a href="https://colab.research.google.com/github/PeterAyad/Flood-Damage-Classification/blob/main/SatelliteImagingProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!fileURL="https://drive.google.com/file/d/1uXhcZxhmoqjaF7_Z-vXNIAWnxmOSMf57/view?usp=sharing"; \
fileName="dataset.zip"; \
fileID=$(echo $fileURL | sed -n 's|.*\(d/\)\([^/]*\)\(\/view\).*|\2|p'); \
echo $fileID; \
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=${fileID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=${fileID}" -O "${fileName}"

In [ ]:
!unzip -q dataset.zip 

In [1]:
%load_ext tensorboard
import torchvision
from torchvision import transforms as T
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm.auto import tqdm 
from IPython.display import Image 
import os
from skimage import io
import time
from sklearn.model_selection import train_test_split
from collections import Counter
import copy
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter
from skimage.transform import resize as io_resize
import cv2
import json
import datetime

In [2]:
RESIZE = (400, 300)
temp_root = "./dataset/"
local_root = "./dataset_resized/"


def resize_and_save(path, resize=RESIZE):
    for img_name in tqdm(os.listdir(os.path.join(temp_root, path))):
        img = cv2.imread(os.path.join(temp_root, path, img_name))
        img = cv2.resize(img, RESIZE)
        cv2.imwrite(os.path.join(local_root, path, img_name), img)

In [3]:
os.makedirs("./dataset_resized/flooded/", exist_ok=True)
os.makedirs("./dataset_resized/non-flooded/", exist_ok=True)

In [19]:
resize_and_save("flooded")
resize_and_save("non-flooded")

  0%|          | 0/461 [00:00<?, ?it/s]

  0%|          | 0/461 [00:00<?, ?it/s]

In [4]:
class T1Dataset(torch.utils.data.Dataset):
    """
    This is to use with array of files and array of labels as an input
    """

    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = io.imread(self.X[idx])
        if self.transform is not None:
            image = self.transform(image)
        if self.y is None:
            return (image, idx)
        return (image, self.y[idx])

In [5]:
transforms = {
    "train": T.Compose(
        [
            T.ToPILImage(),
            # T.RandomResizedCrop(3000),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            # T.Resize((512,512)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "test": T.Compose(
        [
            T.ToPILImage(),
            #  T.CenterCrop(3000),
            #  T.Resize(500),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}


In [6]:
flooded_root = "./dataset_resized/flooded/"
flooded_files = [flooded_root + name for name in os.listdir(flooded_root)]
flooded_y = [1] * len(flooded_files)


In [7]:
non_flooded_root = "./dataset_resized/non-flooded/"
non_flooded_files = [non_flooded_root +
                     name for name in os.listdir(non_flooded_root)]
non_flooded_y = [0] * len(non_flooded_files)


In [8]:
X = flooded_files + non_flooded_files
y = flooded_y + non_flooded_y
print(len(X), len(y))


922 922


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)
print(len(X_train), Counter(y_train), len(X_test), Counter(y_test))

737 Counter({0: 369, 1: 368}) 185 Counter({1: 93, 0: 92})


In [10]:
BATCH_SIZE = 64

unique_labels, counts = np.unique(y_train, return_counts=True)
class_weights = [1 / c for c in counts]
# class_weights[1]*=1.5 # SKEWING THE SAMPLER MORE
sample_weights = [0] * len(y_train)
for idx, lbl in enumerate(y_train):
    sample_weights[idx] = class_weights[lbl]
print(sample_weights)
sampler_train = torch.utils.data.WeightedRandomSampler(
    weights=sample_weights, num_samples=len(sample_weights), replacement=True
)


train_set = T1Dataset(X_train, y_train, transform=transforms["train"])
valid_set = T1Dataset(X_test, y_test, transform=transforms["test"])

train_dataloader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, sampler=sampler_train
)
test_dataloader = torch.utils.data.DataLoader(
    valid_set, batch_size=BATCH_SIZE, shuffle=True
)

[0.0027100271002710027, 0.002717391304347826, 0.0027100271002710027, 0.0027100271002710027, 0.0027100271002710027, 0.0027100271002710027, 0.0027100271002710027, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.0027100271002710027, 0.002717391304347826, 0.002717391304347826, 0.0027100271002710027, 0.0027100271002710027, 0.0027100271002710027, 0.002717391304347826, 0.0027100271002710027, 0.002717391304347826, 0.0027100271002710027, 0.002717391304347826, 0.0027100271002710027, 0.0027100271002710027, 0.002717391304347826, 0.002717391304347826, 0.0027100271002710027, 0.0027100271002710027, 0.0027100271002710027, 0.002717391304347826, 0.0027100271002710027, 0.002717391304347826, 0.0027100271002710027, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.0027100271002710027, 0.002717391304347826, 0.002717391304347826, 0.002717391304347826, 0.002710027

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
model = torchvision.models.resnet18(pretrained=True)
final_layer_in = model.fc.in_features
model.fc = torch.nn.Linear(final_layer_in, 2)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

/home/ubuntu/miniconda3/envs/pytorchenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda3/envs/pytorchenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
def train(model, dataloader, criterion, optimizer, num_epochs):
    # Set the model to train mode
    model.train()

    # Loop over the specified number of epochs
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0

        # Loop over the train dataset
        for inputs, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            # Move inputs and labels to the GPU if available
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Accumulate the running loss and correct predictions
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Compute the epoch loss and accuracy
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        # Log the epoch loss and accuracy
        print(f"Epoch {epoch+1} train Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

    # Return the trained model
    return model

In [17]:
def test(model, dataloader):
    # Set the model to evaluation mode
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    # Loop over the test dataset
    for inputs, labels in tqdm(dataloader, desc="Testing"):
        # Move inputs and labels to the GPU if available
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Accumulate the running loss and correct predictions
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    # Compute the test loss and accuracy
    test_loss = running_loss / len(dataloader.dataset)
    test_acc = running_corrects.double() / len(dataloader.dataset)

    # Log the test loss and accuracy
    print(f"Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}")
    print(classification_report(labels.data.cpu().numpy(), preds.cpu().numpy()))

    # Return the test accuracy
    return test_acc

In [15]:
train(model, train_dataloader, criterion, optimizer, 60)


Epoch 1/60:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1 train Loss: 0.4501, Acc: 0.7775


Epoch 2/60:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 2 train Loss: 0.2553, Acc: 0.9240


Epoch 3/60:   0%|          | 0/12 [00:00<?, ?it/s]

...

Epoch 59/60:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 59 train Loss: 0.0051, Acc: 1.0000


Epoch 60/60:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 60 train Loss: 0.0035, Acc: 1.0000


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [20]:
test(model, test_dataloader)


Testing:   0%|          | 0/3 [00:00<?, ?it/s]

Test Loss: 0.0117, Acc: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        30

    accuracy                           1.00        57
   macro avg       1.00      1.00      1.00        57
weighted avg       1.00      1.00      1.00        57



tensor(1., device='cuda:0', dtype=torch.float64)